# Part 12: Train an Encrypted NN on Encrypted Data

In this notebook, we're going to use all the techniques we've learned thus far to perform neural network training (and prediction) while both the model and the data are encrypted.

In particular, we present our custom Autograd engine which works on encrypted computations.

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Jason Paumier - Github: [@Jasopaum](https://github.com/Jasopaum)
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel)

# Step 1: Create Workers and Toy Data

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [2]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [3]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

# Step 2: Encrypt the Model and Data

Encryption here comes in two steps. Since Secure Multi-Party Computation only works on integers, in order to operate over numbers with decimal points (such as weights and activations), we need to encode all of our numbers using Fixed Precision, which will give us several bits of decimal precision. We do this by calling .fix_precision().

We can then call .share() as we have for other demos, which will encrypt all of the values by sharing them between Alice and Bob. Note that we also set requires_grad to True, which also adds a special autograd method for encrypted data. Indeed, since Secure Multi-Party Computation doesn't work on float values, we can't use the usual PyTorch autograd. Therefore, we need to add a special AutogradTensor node that computes the gradient graph for backpropagation. You can print any of this element to see that it includes an AutogradTensor.

In [4]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [5]:
print(data)

(Wrapper)>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:43749931752 -> bob:51468216654]
	-> (Wrapper)>[PointerTensor | me:81566272589 -> alice:91494764783]
	*crypto provider: james*


# Step 3: Train

And now we can train using simple tensor logic.

In [7]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

tensor([6.1050])
tensor([1.1620])
tensor([0.9690])
tensor([0.9310])
tensor([0.8970])
tensor([0.8920])
tensor([0.8710])
tensor([0.8660])
tensor([0.8410])
tensor([0.8370])
tensor([0.8140])
tensor([0.8040])
tensor([0.7900])
tensor([0.7660])
tensor([0.7680])
tensor([0.7360])
tensor([0.7430])
tensor([0.7350])
tensor([0.7080])
tensor([0.7040])


The loss indeed decreased!

In [7]:
model.fc1.weight.child.child.get().float_precision()

tensor([[ 0.9700,  0.0420],
        [ 0.1300, -0.4020]])

In [8]:
model.fc2.weight.child.child.get().float_precision()

tensor([[1.1320, 0.5690]])

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on Github

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft Github Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for github issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)